In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import os
import numpy as np
# import metview as mv

In [2]:
basePath = "/mnt/qb/goswami/data/era5"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading data")
ds = xr.open_mfdataset(os.path.join(basePath, 'single_pressure_level', '10m_v_component_of_wind', "10m_v_component_of_wind_????.nc"),parallel=True)
print("ds: ")
print(ds.info())
print("fin loading")

loading data
ds: 
xarray.Dataset {
dimensions:
	longitude = 1440 ;
	latitude = 721 ;
	time = 376944 ;

variables:
	float32 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:long_name = longitude ;
	float32 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:long_name = latitude ;
	datetime64[ns] time(time) ;
		time:long_name = time ;
	float32 v10(time, latitude, longitude) ;
		v10:units = m s**-1 ;
		v10:long_name = 10 metre V wind component ;

// global attributes:
	:Conventions = CF-1.6 ;
	:history = 2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1657806762.467146-16033-8-e12dca0a-c8ec-405a-9f94-5781ea6dc0fe.nc /cache/tmp/e12dca0a-c8ec-405a-9f94-5781ea6dc0fe-adaptor.mars.internal-1657806279.235086-16033-9-tmp.grib ;
}None
fin loading


In [27]:
basePath_sfno = "/mnt/qb/work2/goswami0/gkd965/outputs"
file = "v10_SFNO_HalfYearForecast_startfrom_2022-01-01.nc"
# path = os.path.join(basePath, 'single_pressure_level', sfc)
print("loading sfno data")
ds_sfno = xr.open_dataset(os.path.join(basePath_sfno,file), chunks={"step": 4})
print("ds sfno: ")
print(ds_sfno.info())
print("fin loading")

loading sfno data


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/qb/work2/goswami0/gkd965/outputs/v10_SFNO_HalfYearForecast_startfrom_2022-01-01.nc'

In [ ]:
Model = ds.drop([np.datetime64('02-29')], dim='time')

In [3]:
ds['hourofyear'] = xr.DataArray(ds.time.dt.strftime('%m-%d %H'), coords=ds.time.coords)
ds_hourofyear = ds.groupby("hourofyear")
mean_temps = ds_hourofyear.mean()

In [4]:
mean_temps.info()

xarray.Dataset {
dimensions:
	longitude = 1440 ;
	latitude = 721 ;
	hourofyear = 8784 ;

variables:
	float32 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:long_name = longitude ;
	float32 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:long_name = latitude ;
	float32 v10(hourofyear, latitude, longitude) ;
		v10:units = m s**-1 ;
		v10:long_name = 10 metre V wind component ;
	object hourofyear(hourofyear) ;

// global attributes:
	:Conventions = CF-1.6 ;
	:history = 2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmwf/mars-client/bin/grib_to_netcdf.bin -S param -o /cache/data1/adaptor.mars.internal-1657806762.467146-16033-8-e12dca0a-c8ec-405a-9f94-5781ea6dc0fe.nc /cache/tmp/e12dca0a-c8ec-405a-9f94-5781ea6dc0fe-adaptor.mars.internal-1657806279.235086-16033-9-tmp.grib ;
}

In [10]:
Model = mean_temps.to_array(dim='v10').squeeze()

In [14]:
Model

<xarray.DataArray (hourofyear: 8784, latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(8784, 721, 1440), dtype=float32, chunksize=(1, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * hourofyear  (hourofyear) object '01-01 00' '01-01 01' ... '12-31 23'
    v10         <U3 'v10'
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [22]:
# hourofyear for leapyear
start_feb_29 = 24*(31+28)
end_feb_29 = start_feb_29 + 24
end_feb_29 - start_feb_29

Model_noleapyear = xr.concat((Model[:start_feb_29,:,:], Model[end_feb_29:,:,:]),dim='hourofyear')

In [24]:
Model_noleapyear[start_feb_29 - 1]

<xarray.DataArray (latitude: 721, longitude: 1440)>
dask.array<getitem, shape=(721, 1440), dtype=float32, chunksize=(721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * longitude   (longitude) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude    (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    hourofyear  <U8 '02-28 23'
    v10         <U3 'v10'
Attributes:
    Conventions:  CF-1.6
    history:      2022-07-14 14:01:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [25]:
Model_6hr = Model_noleapyear[::6,:,:]
Model_6hr.shape

(1460, 721, 1440)

In [ ]:
Ref = mean_temps.to_array(dim='2m_temperature')
Model = ds_sfno.to_array(dim='2t')

In [ ]:
RMSE = 1 - Model / Ref